In [1]:
# Before we begin, we supress deprecation warnings resulting from nltk on Kaggle
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [2]:
import pandas as pd
tweets = pd.read_csv("C:/Users/pk/Desktop/projet/analyse/out_data/TweetIphone.csv")
list(tweets.columns.values)

['Unnamed: 0',
 'date',
 'text',
 'Comment',
 'Word_Tok',
 'CommentAferPreproc',
 'sentiment']

In [3]:
tweets.head()

,Unnamed: 0,date,text,Comment,Word_Tok,CommentAferPreproc,sentiment
0,0,2021-04-01 23:59:32,"b""RT @_PLICE: \xf0\x9f\xa4\xa3\xf0\x9f\xa4\xa3...","b""rt @_plice: \xf0\x9f\xa4\xa3\xf0\x9f\xa4\xa3...","['b', 'rt', '_plice', 'xf0', 'x9f', 'xa4', 'xa...",b rt _plice xf0 x9f xa4 xa3 xf0 x9f xa4 xa3 xf...,Neutral
1,1,2021-04-01 23:59:32,b'RT @twinkologian: sza the type to say she go...,b'rt @twinkologian: sza the type to say she go...,"['b', 'rt', 'twinkologian', 'sza', 'the', 'typ...",b rt twinkologian sza type say got iphone 15 x...,Neutral
2,2,2021-04-01 23:59:29,b'Check out this product from the Apple Store:...,b'check out this product from the apple store:...,"['b', 'check', 'out', 'this', 'product', 'from...",b check product apple store n nhttps co xozusg...,Neutral
3,3,2021-04-01 23:59:28,b'I\xe2\x80\x99ve had a great birthday so far!...,b'i\xe2\x80\x99ve had a great birthday so far!...,"['b', 'i', 'xe2', 'x80', 'x99ve', 'had', 'a', ...",b xe2 x80 x99ve great birthday far mom delicio...,Positive
4,4,2021-04-01 23:59:21,b'I go on vacation for the first time in well ...,b'i go on vacation for the first time in well ...,"['b', 'i', 'go', 'on', 'vacation', 'for', 'the...",b go vacation first time well year think would...,Positive


In [4]:
sentiment_counts = tweets.sentiment.value_counts()
print(sentiment_counts)

Neutral     4981
Positive    3479
Negative    1771
Name: sentiment, dtype: int64


In [5]:
import numpy as np
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(ngram_range=(1,2))

In [6]:
vectorized_data = count_vectorizer.fit_transform(tweets.text)
indexed_data = hstack((np.array(range(0,vectorized_data.shape[0]))[:,None], vectorized_data))

In [7]:
def sentiment2target(sentiment):
    return {
        'Negative': 0,
        'Neutral': 1,
        'Positive' : 2
    }[sentiment]
targets = tweets.sentiment.apply(sentiment2target)

In [8]:
from sklearn.model_selection import train_test_split
data_train, data_test, targets_train, targets_test = train_test_split(indexed_data, targets, test_size=0.4, random_state=0)
data_train_index = data_train[:,0]
data_train = data_train[:,1:]
data_test_index = data_test[:,0]
data_test = data_test[:,1:]

# SVM Model

In [10]:
from sklearn import svm
from sklearn import metrics
from sklearn.multiclass import OneVsRestClassifier
clf = OneVsRestClassifier(svm.SVC(gamma=0.01, C=100., probability=True, class_weight='balanced', kernel='linear'))
clf_output = clf.fit(data_train, targets_train)
clf.score(data_test, targets_test)
pre=clf.predict(data_test)
acc=metrics.accuracy_score(pre, targets_test)
print(str("{:04.2f}".format(acc*100))+'%')

94.01%


# Naive bayes Model 

In [11]:
from sklearn.naive_bayes import MultinomialNB
mnb=MultinomialNB()
mnb.fit(data_train,targets_train)
preticted=mnb.predict(data_test)
accuracy=metrics.accuracy_score(preticted, targets_test)
print(str("{:04.2f}".format(accuracy*100))+'%')

89.76%


# Decision tree Model 

In [12]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier()
dt.fit(data_train,targets_train)
preticted3=dt.predict(data_test)
accuracy3=metrics.accuracy_score(preticted3, targets_test)
print(str("{:04.2f}".format(accuracy3*100))+'%')

92.13%


# Model Random Forest

In [13]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=40)
rf.fit(data_train,targets_train)
preticted4=rf.predict(data_test)
accuracy4=metrics.accuracy_score(preticted4, targets_test)
print(str("{:04.2f}".format(accuracy4*100))+'%')

90.89%
